In [167]:
from yahooquery import Ticker
import pandas as pd
from datetime import datetime, date
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="yahooquery")

def get_raw_yahooquery(symbol_list, start_date, import_time):
    try:
        # Attempt to fetch data from Yahoo Finance
        data = Ticker(symbol_list)
        current_date = date.today()
       
        # Fetch historical data
        hist_data = data.history(start=start_date, end=current_date, interval="1d")
        # print(hist_data)
        
        # Reshape the data to include a 'Ticker' column
        # hist_data.reset_index(inplace=True)  # Reset index to make Ticker and Date columns
        hist_data = hist_data.reset_index()

        # # Select only required columns
        columns_to_select = ["date", "symbol", "open", "high", "low", "close", "volume"]
        hist_data = hist_data[columns_to_select]
        hist_data["date"] = pd.to_datetime(hist_data["date"]).dt.tz_localize(None)
        # Add a new field for the current datetime
        hist_data["import_time"] = pd.to_datetime(import_time).tz_localize(None)

        # Return the processed data
        return hist_data
    
    except Exception as e:
        # Handle any exception that occurred during the execution
        print(f"An error occurred: {e}")
        
        # Return an empty DataFrame with the same columns as the expected result
        return pd.DataFrame(columns=["date", "symbol", "open", "high", "low", "close", "volume", "import_time"])  



In [176]:
from yahooquery import Ticker
import pandas as pd
from datetime import date

# Define your symbol list and date range
symbol_list = ['HCVIU', 'MSFT']  # Example symbols
start_date = '2025-01-08'  # Example start date
current_date = date.today()

# Fetch data from Yahoo Finance
data = Ticker(symbol_list)

# Fetch historical data
hist_data = data.history(start=start_date, end=current_date, interval="1d")

# Reset index to make 'date' and 'symbol' accessible as columns
hist_data = hist_data.reset_index()

# Select only required columns
columns_to_select = ["date", "symbol", "open", "high", "low", "close", "volume"]
hist_data = hist_data[columns_to_select]

# Normalize 'date' column to remove timezone awareness
hist_data["date"] = hist_data["date"].astype(str).str.slice(0, 10)
hist_data["date"] = pd.to_datetime(hist_data["date"])


# Print the type and data for verification
print(type(hist_data))
print(hist_data)

# Return or process the data as needed
# return hist_data


<class 'pandas.core.frame.DataFrame'>
        date symbol        open        high         low       close    volume
0 2025-01-08  HCVIU   10.570000   10.570000   10.570000   10.570000         0
1 2025-01-08   MSFT  423.459991  426.970001  421.540009  424.559998  15054600


In [168]:
symbol_list=['IVA', 'TBLT', 'TRNO', 'IONM', 'EGRX', 'OCAX', 'MBNKP', 'BRKHU', 'BNIX', 'SRDX']
get_raw_yahooquery(symbol_list, '2024-11-15', datetime.now())

,date,symbol,open,high,low,close,volume,import_time
0,2024-11-15,IVA,2.750000,2.750000,2.631000,2.720000,12500,2025-01-10 01:29:21.899691
1,2024-11-18,IVA,2.640000,2.730000,2.630000,2.730000,19900,2025-01-10 01:29:21.899691
2,2024-11-19,IVA,2.630000,2.767000,2.630000,2.740000,17700,2025-01-10 01:29:21.899691
3,2024-11-20,IVA,2.700000,2.740000,2.630000,2.740000,6700,2025-01-10 01:29:21.899691
4,2024-11-21,IVA,2.640000,2.670000,2.510000,2.660000,14200,2025-01-10 01:29:21.899691
...,...,...,...,...,...,...,...,...
350,2025-01-02,SRDX,39.660000,40.000000,39.500000,39.680000,299300,2025-01-10 01:29:21.899691
351,2025-01-03,SRDX,39.750000,39.980000,39.500000,39.520000,167500,2025-01-10 01:29:21.899691
352,2025-01-06,SRDX,39.509998,40.209999,39.509998,39.709999,224100,2025-01-10 01:29:21.899691
353,2025-01-07,SRDX,39.709999,40.070000,38.820000,38.980000,221400,2025-01-10 01:29:21.899691


In [158]:
import yfinance as yf
import pandas as pd
from datetime import datetime

def get_raw_yfinance(symbol_list, start_date, import_time):
    try:
        current_date = date.today()
        # Fetch historical data
        hist_data = yf.download(
            tickers=symbol_list,
            start=start_date,
            end=current_date,
            interval="1d",
            group_by="ticker",  # Keep data grouped by ticker if multiple symbols are provided
            auto_adjust=False,  # Keep original values without adjustment
        )
        
        # Check if the data needs reshaping for multiple tickers
        hist_data = hist_data.stack(level=0, future_stack=True).reset_index()   

        hist_data.rename(columns={"Date": "date", "Ticker":"symbol", "Open": "open", "High": "high", 
                                  "Low": "low", "Close": "close", "Volume": "volume"}, inplace=True)
        

        # hist_data.columns = [col.lower() for col in hist_data.columns]
       
        # Select only required columns
        columns_to_select = ["date", "symbol", "open", "high", "low", "close", "volume"]
        hist_data = hist_data[columns_to_select]
        
        # Ensure the date is in datetime format
        hist_data["date"] = pd.to_datetime(hist_data["date"])
        
        # Add an import_time column
        hist_data["import_time"] = pd.to_datetime(import_time)
        # print(hist_data)
        # # Return the processed data
        return hist_data
    
    except Exception as e:
        # Handle any exceptions
        print(f"An error occurred: {e}")        
        # Return an empty DataFrame with the same columns as the expected result
        return pd.DataFrame(columns=["date", "symbol", "open", "high", "low", "close", "volume", "import_time"])


In [149]:
# symbol_list=['C']
# get_raw_yfinance(symbol_list, '2025-1-1', datetime.now())